In [ ]:
gibbs_sampler <- function(X, Y, alpha_init = rep(0,dim(X)[2]), sigma2_init = 1,
                          delta = 10, lambda = 10, mu = rep(0,dim(X)[2]),
                          Omega = diag(dim(X)[2]), MCsamplesize = 1000){

    # dims of X
  n <- dim(X)[1] # sample size
  p <- dim(X)[2] # number of covariates in linear regression model/alpha-parameters

  # set starting values for parameter chains
    # variables where generated values will be saved
  alpha_chain      <- matrix(NA, MCsamplesize, p)
  sigma2_chain     <- rep   (NA, MCsamplesize)
    # first values for both chains
  alpha_chain[1,]  <- alpha_init  # [0] * p
  sigma2_chain[1]  <- sigma2_init  # [1]

  # create parameter chain
  for (i in 2:(MCsamplesize)){

    alpha_chain[i,]  <- alpha_chain[i-1,]
    sigma2_chain[i]  <- sigma2_chain[i-1]

    for (j in 1:p){

      s_j <- 1 / (Omega[j, j] + sigma2_chain[i]^(-1) * sum(X[, j]^2))
      m_j <- s_j*(mu[j]*Omega[j, j]
                  - sum(Omega[j, -j]*(alpha_chain[i, -j] - mu[-j]))
                  + sigma2_chain[i]^(-1)*sum(X[, j]*(Y - X[, -j]%*%alpha_chain[i, -j])))
      alpha_chain[i,j] <- rnorm(1, m_j, sqrt(s_j)) # rnorm accepts sd, not var

    }

    sigma2_chain[i] <- 1 / rgamma(n=1, shape=delta + n/2,
                                  rate=lambda + 1/2*sum( (Y - X%*%alpha_chain[i, ])^2 ))
  }

  return(list("alpha" = alpha_chain, "sigma2" = sigma2_chain))

}

In [8]:
gibbs_sampler_2 <- function(Y, mu_init=runif(2,-10,10), sigma1=1, sigma2=1, rho=0.05,
                            R=1000){
    # extract the number of datapoints
  n <- dim(Y)[1]

    # Prepare the matrix or array to store the chain
  mu_chain      <- matrix(NA, R, 2)
    # set initial values
  mu_chain[1,]  <- mu_init
    
    # this data could be in a loop, but this way saves a bit of space
  # same for all iterations
  sd1 <- sqrt(sigma2^2*(1-rho^2) / n)
  sd2 <- sqrt(sigma1^2*(1-rho^2) / n)
  y.mean1 <- mean(Y[,1])
  y.mean2 <- mean(Y[,2])
    
  # create parameter chain
  for (i in 2:(R)){
      # UPDATE ALL VARIABLES ONE AFTER ANOTHER
    mu_chain[i, 1] = rnorm(1, 
                           mean=(y.mean1 + rho*sigma1/sigma2 * (mu_chain[i-1,2] - y.mean2)),
                           sd=sd1)
    mu_chain[i, 2] = rnorm(1, 
                           mean=(y.mean2 + rho*sigma2/sigma2 * (mu_chain[i,1] - y.mean1)),
                           sd=sd2)
  }
      
  return(mu_chain)
}

In [4]:
set.seed(42)
mu_real = c(6,2)
sigma1 = sqrt(2)
sigma2 = sqrt(0.5)
rho = 0.05
n = 100
Sig = matrix(c(sigma1^2, sigma1*sigma2*rho, 
               sigma1*sigma2*rho, sigma2^2), nrow = 2, ncol = 2)

Y <- MASS::mvrnorm(n, mu_real, Sig)
R1 = 100
R2 = 100000
mu_chain_1 <- gibbs_sampler_2(Y, sigma1, sigma2, rho, R=R1)

ERROR: Error in gibbs_sampler_2(Y, sigma1, sigma2, rho, R = R1): не могу найти функцию "gibbs_sampler_2"


In [5]:
p1 <- c(0.5, 0.4, 0.1) 
p2 <- c(0.3, 0.4, 0.3) 
p3 <- c(0.2, 0.3, 0.5)
set.seed(42) 
N <- 10^2

markov_chain <- function(N,pi0=c(1,1,1)){
    t <- numeric(N)
    
    t[1] <- sample(c(1,2,3), size=1, prob=pi0)
    for (i in 2:(N)){
        if (t[i-1]==1){ 
            p <- p1 }
        else if (t[i-1]==2){ 
            p <- p2 }
        else { 
            p <- p3 }
        t[i] <- sample(c(1,2,3), size=1, prob = p)
    }
    return(t)
}

chain <- markov_chain(N,c(0.4,0.5,0.1))

In [ ]:
myrgamma <- function(R, alpha){ 
    alpha_1 <- exp(1)/(alpha + exp(1))
    out <- rep(0, R) 
    for (i in 1:R){
        
        cond_accpt <- FALSE 
        while(cond_accpt == FALSE){
            u_chooseQ  <- runif(1)
            u_PIT      <- runif(1)
            u_accpt    <- runif(1)
            x <- (u_chooseQ < alpha_1) * u_PIT^(1/alpha) +
                 (u_chooseQ >= alpha_1) * (1 - log(u_PIT))
            if (u_accpt <= (exp(-x)*(x<=1)+x^(alpha-1)*(x>1))){
                cond_accpt <- TRUE
            } 
        }
        out[i] <- x
    } 
    
    return(out)
}

In [8]:
MH <- function(y, Sigma, mu_init=runif(2,-10,10), R=1000){ 
    n <- dim(y)[1]
    
    # Prepare Chain
    mu_chain <- matrix(NA, R, 2) 
    
    # init values
    mu_chain[1,] <- mu_init

    # Precompute
    y.mean1 <- mean(y[,1]) 
    y.mean2 <- mean(y[,2])
    y.mean <- c(y.mean1, y.mean2)
    Sigma_inv = solve(Sigma)

    for (i in 2:(R)){
        
        # old value
        old <- mu_chain[i-1,]
        # porposal
        new <- mvtnorm::rmvnorm(n=1, mean=old, sigma=Sigma)

        # Compute a
        v1 = matrix(new-y.mean, nrow=2, ncol=1) 
        v2 = matrix(old-y.mean, nrow=2, ncol=1) 
        term1 = t(v1) %*% Sigma_inv %*% v1 
        term2 = t(v2) %*% Sigma_inv %*% v2
        
        rate = exp(-n/2 * (term1-term2)) 
        
        a <- min(1, rate)
        
        if (a > runif(1)){ 
            mu_chain[i,] <- new
        } 
        else{
            mu_chain[i,] <- old 
        }
    }
    return(mu_chain) 
}

In [10]:
# MH(Y, Sig, R=100)